## Setting LLM Model

In [24]:
import nest_asyncio

nest_asyncio.apply()

In [15]:
from dotenv import load_dotenv
import os

load_dotenv()
#print("Open AI - ",os.getenv("LITELLM_URL"),os.getenv("OPENAI_API_MODEL"), os.getenv("OPENAI_API_EMBEDDING"))
#print("OLLAMA  - ",os.getenv("OLLAMA_URL"),os.getenv("OLLAMA_MODEL"))
#print("Local OLLAMA - ",os.getenv("OLLAMA_LOCAL_URL"),os.getenv("OLLAMA_LOCAL_MODEL"))

True

In [16]:
from llama_index.core import Settings

### RUN LLM AS OLLAMA 

In [ ]:
#install OPEN AI LLM, skip if already installed
!pipenv install llama-index-llms-ollama

In [26]:
from llama_index.llms.ollama import Ollama
api_base=os.getenv("OLLAMA_LOCAL_URL")
model=os.getenv("OLLAMA_LOCAL_MODEL")
llm = Ollama(model=model, base_url=api_base,request_timeout=120.0)

# use remote ollam
"""
api_base=os.getenv("OLLAMA_URL")
model=os.getenv("OLLAMA_MODEL")
llm = Ollama(model=model, base_url=api_base,request_timeout=180.0)
"""
#test run
response = llm.complete("What is the capital of France?")
print(response)

The capital of France is **Paris**. 🇫🇷  



### RUN LLM AS OPEN AI 

In [ ]:
#install OPEN AI LLM, skip if already installed
!pipenv install llama-index-llms-openai

In [17]:
from llama_index.llms.openai import OpenAI
api_base=os.getenv("LITELLM_URL")
model=os.getenv("OPENAI_API_MODEL")

Settings.llm = OpenAI(
    model=model,
    api_base = api_base,
    temperature=0.3
)

#resp = Settings.llm.complete("What is the capital of France?")
#print(resp)

## Setting Embedding Model

In [27]:
# use open AI embedding
from llama_index.embeddings.openai import OpenAIEmbedding
api_base=os.getenv("LITELLM_URL")
embedding_model=os.getenv("OPENAI_API_EMBEDDING")

Settings.embed_model = OpenAIEmbedding(
    model_name=embedding_model,
    api_base = api_base,
)
#embed_text = Settings.embed_model.get_text_embedding("hello")
#print(f"{len(embed_text)}, {embed_text}")

In [35]:
# use other embedding
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)
#embed_text = Settings.embed_model.get_text_embedding("hello")
#print(f"{len(embed_text)}, {embed_text}")

In [28]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
documents = SimpleDirectoryReader("./data/paul").load_data()

In [36]:
import logging
import sys

#logging.basicConfig(stream=sys.stdout, level=logging.INFO)
#logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from IPython.display import Markdown, display

### Node
Nodes represent "chunks" of source Documents, whether that is a text chunk, an image, or more. They also contain metadata and relationship information with other nodes and index structures

In [37]:
from llama_index.core.node_parser import SentenceSplitter

parser = SentenceSplitter(chunk_size=1024, chunk_overlap=20)
nodes = parser.get_nodes_from_documents(documents)

In [38]:
print(f"Total Nodes: {len(nodes)}")
for node in nodes[:5]:
    print(f"Node ID: {node.node_id}, {node.text[0:50]}")

Total Nodes: 18
Node ID: 35b4b350-e385-4c06-969d-d11ebaeeaece, What I Worked On

February 2021

Before college th
Node ID: 147a7851-770f-4295-a966-53f79b798893, It seemed only a matter of time before we'd have M
Node ID: 4339b78b-f145-4566-b9fe-38a8c0b9bead, There were some surplus Xerox Dandelions floating 
Node ID: 80918e0f-b0bb-4bfb-94f9-395d2b27e172, This was now only weeks away. My nice landlady let
Node ID: d45484e8-5de2-4b05-a559-c416aaf3d50f, Our teacher, professor Ulivi, was a nice guy. He c


### Node with MetaData Extractor

In [39]:
from llama_index.core.extractors import (
    TitleExtractor,
    QuestionsAnsweredExtractor,
)
from llama_index.core import VectorStoreIndex

title_extractor = TitleExtractor(nodes=5)
qa_extractor = QuestionsAnsweredExtractor(questions=3)

In [ ]:
# assume documents are defined -> extract nodes
from llama_index.core.ingestion import IngestionPipeline

pipeline = IngestionPipeline(
    transformations=[parser, title_extractor, qa_extractor]
)
nodes = pipeline.run(
    documents=documents,
    in_place=True,
    show_progress=True,
)

In [59]:
for key,value in nodes[0].metadata.items():
    if(key in ["document_title","questions_this_excerpt_can_answer"]):
        print(f"{key}:{value}")

document_title:"From Code to Canvas: A Journey of Discovery Through Computing, Art, and Philosophy"
questions_this_excerpt_can_answer:Based on the provided context from the excerpt of Paul Graham's essay, here are three specific questions that can be answered:

1. **What programming language did Paul Graham first use when he began programming on the IBM 1401, and what was the method of input for the programs?**
   - This question focuses on the specific programming language (Fortran) and the unique method of input (punch cards) that Graham used, which is a detail that may not be widely known or found elsewhere.

2. **What was the significance of the TRS-80 in Paul Graham's programming journey, and what types of programs did he create with it?**
   - This question highlights the importance of the TRS-80 in Graham's development as a programmer and specifies the kinds of programs he wrote, such as simple games and a word processor, which provides insight into his early programming experie

In [ ]:
index = VectorStoreIndex(nodes)
query_engine = index.as_query_engine(response_mode="tree_summarize")
print(query_engine.query("Who is Paul Graha?"))

### Semantic Chunker
Semantic splitter adaptively picks the breakpoint in-between sentences using embedding similarity. This ensures that a "chunk" contains sentences that are semantically related to each other.

In [ ]:
from llama_index.core.node_parser import SemanticSplitterNodeParser

In [ ]:
splitter = SemanticSplitterNodeParser(
buffer_size=1, breakpoint_percentile_threshold=95, embed_model=Settings.embed_model
)

In [ ]:
nodes = splitter.get_nodes_from_documents(documents)

In [ ]:
print(f"Total Nodes: {len(nodes)}")
for node in nodes[:5]:
    print(f"Node ID: {node.node_id}, {node.get_content()[:100]}")

In [ ]:
index = VectorStoreIndex(nodes)
query_engine = index.as_query_engine(response_mode="tree_summarize")
print(query_engine.query("Who is Paul Graha?"))

### Markdown Node Parser

In [ ]:
from llama_index.core.node_parser import MarkdownNodeParser
markdown_docs = SimpleDirectoryReader("./data/markdown").load_data()

In [ ]:
print(markdown_docs[2].text[0:500])

In [ ]:
parser = MarkdownNodeParser()
nodes = parser.get_nodes_from_documents(markdown_docs)

In [ ]:
print(f"Total Nodes: {len(nodes)}")
for node in nodes[:5]:
    print(node)

In [ ]:
from llama_index.core import VectorStoreIndex
index = VectorStoreIndex(nodes)
query_engine = index.as_query_engine()
print(query_engine.query("Give me types of KPI?"))